In [1]:
#IPython magic to reload libraries
%load_ext autoreload
%autoreload 2

# Extra code so Juypter doesn't crash on mac os
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import sys
sys.path.insert(0, 'utils')  # noqa

import numpy as np
import gym
import ntm
import interfaces

In [3]:
# Training Params
train_iter = 100
batch_size = 1


In [4]:
# Task
env = gym.make('Copy-v0')
env = env.unwrapped
print(env.action_space)
print(env.observation_space)


Tuple(Discrete(2), Discrete(2), Discrete(5))
Discrete(6)


/Users/zachyamaoka/opt/anaconda3/envs/tf2_robo_research/lib/python3.7/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [5]:
# Controller 
Controller = ntm.NTM()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tape_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
memory_input (InputLayer)       [(None, 10)]         0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 11)           0           tape_input[0][0]                 
                                                                 memory_input[0][0]               
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           384         concatenate[0][0]            

In [6]:
#Over Fit on single char
output = Controller.run(tape_input=1)
print(output)
Controller.update(output_target=4)

(0, 1, 1)
Loss:  [1.4307102]


In [7]:
output_control_cmds = ["no","yes"]
input_control_cmds = ["left","right"]
int_to_char = ["A","B","C","D","E","F"]
render_env = False
for i in range(train_iter):
# for i in range(1):

    # Collect experince using current policy/controller
    for j in range(batch_size):
        
        episode_steps = 0
        observation = env.reset()
        if render_env: env.render()

        while True:
            episode_steps += 1
            
            #Run RL-NTM controller and get outputs
            outputs = Controller.run(tape_input=observation)

            output_content = outputs[0] # Which character to write (ignored if the above sub-action is 0) 
            output_control = outputs[1] #  Whether to write to the output tape ["no","yes"] = [0,1]
            input_control = outputs[2] # Direction to move the read head ["left","right"] = [0,1]
           
            actions = (input_control, output_control, output_content)
            
            print("Policy - obs: {} action: {}, {}, {}".format(int_to_char[observation],
                  int_to_char[output_content],
                  output_control_cmds[output_control],
                  input_control_cmds[input_control]))

            #Get output target before updating enviroment
            target = env.target[env.write_head_position]
            observation, reward, done, info = env.step(actions)
            if render_env: env.render()
                
            if output_control_cmds[output_control] == "yes":
                Controller.update(output_target=target)
                
            if done:
                Controller.backup()
                done = False
                break
        

Policy - obs: C action: A, yes, right
Loss:  [1.1708977]
Policy - obs: E action: C, yes, right
Loss:  [1.4809636]
Policy - obs: A action: A, yes, right
Loss:  [2.01573]
Policy - obs: D action: C, yes, right
Loss:  [2.041598]
Policy - obs: E action: C, yes, right
Loss:  [1.4207302]
Policy - obs: D action: A, yes, right
Loss:  [1.8481375]
Policy - obs: B action: B, yes, right
Loss:  [2.0037434]
Policy - obs: D action: D, yes, right
Loss:  [1.7003218]
Policy - obs: D action: C, yes, right
Loss:  [1.6063377]
Policy - obs: C action: C, yes, right
Loss:  [1.3801754]
Policy - obs: B action: D, yes, right
Loss:  [1.872267]
Policy - obs: D action: E, yes, right
Loss:  [1.5729563]
Policy - obs: A action: D, yes, right
Loss:  [1.8047199]
Policy - obs: E action: C, yes, right
Loss:  [1.5008891]
Policy - obs: D action: B, yes, right
Loss:  [1.5327643]
Policy - obs: B action: E, yes, right
Loss:  [1.8022217]
Policy - obs: C action: E, yes, right
Loss:  [1.4254212]
Policy - obs: B action: E, yes, rig